# Pangloss Performance Benchmark

In [4]:
from pangloss import BackgroundCatalog, ForegroundCatalog, \
    TrueHaloMassDistribution, Kappamap, Shearmap

ITERATIONS = 4
RADIUS = 2.0

# initialize background and foreground
B = BackgroundCatalog(N=10.0, domain=[1.5, 1.4, -1.5, -1.4], field=[0, 0, 0, 0])
F = ForegroundCatalog.guo()
F.set_mass_prior(TrueHaloMassDistribution())

# initialize maps from Hilbert et al 2009
K = Kappamap.example()
S = Shearmap.example()

# run monte carlo samples
def pangloss_benchmark():
    for _ in xrange(ITERATIONS):
        F.draw_halo_masses()
        B.drill_lightcones(radius=RADIUS, foreground=F, smooth_corr=False)
        B.lens_by_map(K, S)
        B.lens_by_halos(lookup_table=False, smooth_corr=False, relevance_lim=0)
        B.halo_mass_log_likelihood()

['$PANGLOSS_DIR/calib/Millennium/catalog_example.txt']
['$PANGLOSS_DIR/calib/Millennium/GGL_los*.fits']
         45253159 function calls (44562463 primitive calls) in 164.629 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.031    0.031  164.629  164.629 <ipython-input-4-3d073afda2ed>:18(pangloss_benchmark)
        1    0.000    0.000  164.629  164.629 <string>:1(<module>)
    90817    0.073    0.000    0.104    0.000 __init__.py:163(iteritems)
     5764    0.006    0.000    0.007    0.000 __init__.py:169(itervalues)
       12    0.000    0.000    0.001    0.000 _methods.py:122(_std)
     2880    0.010    0.000    0.659    0.000 _methods.py:25(_amax)
     2880    0.010    0.000    0.675    0.000 _methods.py:28(_amin)
    17296    0.022    0.000    2.586    0.000 _methods.py:31(_sum)
    56312    0.052    0.000    0.473    0.000 _methods.py:37(_any)
     2908    0.003    0.000    0.030    0.000 _methods.py:40(_

# MassInference Performance Benchmark

In [2]:
from massinference.angle import Angle
from massinference.catalog import SourceCatalogFactory, FastSampleHaloCatalogFactory, \
    MutableHaloMassCatalog, SourceCatalog, HaloCatalog
from massinference.distribution import MassPrior
from massinference.inference import log_likelihood
from massinference.lenser import MapLenser
from massinference.lightcone import LightconeManager
from massinference.map import KappaMap, ShearMap
from massinference.plot import Limits

import cProfile

ITERATIONS = 4
RADIUS = 2.0

# run parameters
sigma_e = 0.2
random_seed = 1
source_density = 10.0
limits = Limits(Angle.from_degree(1.5), Angle.from_degree(1.4),
                Angle.from_degree(-1.5), Angle.from_degree(-1.4))
limits_with_perimeter = limits.add_perimeter(Angle.from_arcmin(RADIUS))

# make a mock WL catalog, of observed, lensed, galaxy ellipticities:
source_factory = SourceCatalogFactory(limits, source_density, sigma_e)
source_catalog = source_factory.generate()
max_z = source_catalog.dataframe[SourceCatalog.Z].max()

e1, e2 = MapLenser(KappaMap.default(), ShearMap.default()).lens(source_catalog)
base_halo_catalog = MutableHaloMassCatalog.default(limits, max_z)
mass_prior = MassPrior(base_halo_catalog.dataframe[HaloCatalog.HALO_MASS].as_matrix())
halo_catalog_factory = FastSampleHaloCatalogFactory(base_halo_catalog,
                                                    mass_prior, random_seed)
lightcone_manager = LightconeManager(source_catalog, halo_catalog_factory, RADIUS)


def mass_inference_benchmark():
    predictions = lightcone_manager.run(ITERATIONS)
    log_likelihood(predictions, e1, e2, sigma_e)

cProfile.run('mass_inference_benchmark()')

         52642 function calls (52626 primitive calls) in 0.568 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.568    0.568 <ipython-input-2-4bb831be9e3f>:36(mass_inference_benchmark)
        1    0.000    0.000    0.568    0.568 <string>:1(<module>)
        8    0.000    0.000    0.000    0.000 _internal.py:227(__init__)
        8    0.000    0.000    0.000    0.000 _internal.py:252(get_data)
        2    0.000    0.000    0.000    0.000 _methods.py:122(_std)
     4328    0.002    0.000    0.017    0.000 _methods.py:31(_sum)
     4320    0.002    0.000    0.018    0.000 _methods.py:37(_any)
        3    0.000    0.000    0.000    0.000 _methods.py:43(_count_reduce_items)
        1    0.000    0.000    0.000    0.000 _methods.py:53(_mean)
        2    0.000    0.000    0.000    0.000 _methods.py:76(_var)
       16    0.000    0.000    0.000    0.000 base.py:1275(<lambda>)
        8    0.000 

# Conclusions
- MassInference outperforms Pangloss by 290x
- Only drilling once and removing duplication at the lightcone level has led to a big performance boost, as expected
- MassInference spends ~80% of time in lensing code (lensing.bmo_f, lensing.bmo_g)
    - This code performs a long sequence of numerical operations on large arrays, would be prime for GPU acceleration
    - We could also try bin-caching
- The iterative part of the calculation is embarassingly parallel and should scale linearly over cluster
    - mpi4py
- Pangloss spends lots of time garbage collecting
    - If this pops up in MassInference we can look into making an objectpool, resuing objects etc